# Overview Materi

Jelaskan perbedaan singkat antara grid, randomized, bayesian search cv dengan optuna menurut pemahamanmu

source: https://www.youtube.com/watch?v=t-INgABWULw

# Import Data & Libraries

In [1]:
# jalankan hanya sekali
!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 10.1 MB/s eta 0:00:00


In [2]:
# import library yang dibutuhkan di sini
import seaborn as sns
import pandas as pd

In [3]:
df = sns.load_dataset('iris')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [4]:
df.tail()

,sepal_length,sepal_width,petal_length,petal_width,species
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


# Data Preprocessing

In [15]:
# ubah variabel kategorik ke numerik
iris = pd.get_dummies(df)
iris[ ['species_setosa', 'species_versicolor', 'species_virginica'] ] = iris[ ['species_setosa', 'species_versicolor', 'species_virginica'] ].astype(int)
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species_setosa,species_versicolor,species_virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


In [16]:
iris.tail()

,sepal_length,sepal_width,petal_length,petal_width,species_setosa,species_versicolor,species_virginica
145,6.7,3.0,5.2,2.3,0,0,1
146,6.3,2.5,5.0,1.9,0,0,1
147,6.5,3.0,5.2,2.0,0,0,1
148,6.2,3.4,5.4,2.3,0,0,1
149,5.9,3.0,5.1,1.8,0,0,1


In [22]:
# subsetting peubah
X = iris.drop(['species_setosa', 'species_versicolor', 'species_virginica'], axis=1)
y = iris[['species_setosa', 'species_versicolor', 'species_virginica']]
X.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [23]:
y.head()

,species_setosa,species_versicolor,species_virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [32]:
X.shape

(150, 4)

In [29]:
y.shape

(150, 3)

# Dataset Splitting

In [30]:
# split dengan rasio 80:20
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=74)
X_train.shape

(120, 4)

In [33]:
X_test.shape

(30, 4)

# Base Model Random Forest

In [35]:
# gunakan random forest classifier
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=92)
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=92)

In [38]:
y_pred = rfc.predict(X_test)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [39]:
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.3f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.3f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted'):.3f}")
print(classification_report(y_test, y_pred))

Accuracy: 1.000
Precision: 1.000
Recall: 1.000
F1 Score: 1.000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        11
           2       1.00      1.00      1.00         9

   micro avg       1.00      1.00      1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30
 samples avg       1.00      1.00      1.00        30



# Optuna

In [72]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 10, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 32)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 32)

    model = RandomForestClassifier(n_estimators = n_estimators,
                                   max_depth = max_depth,
                                   min_samples_split = min_samples_split,
                                   min_samples_leaf = min_samples_leaf)

    scorer = make_scorer(precision_score, average='weighted')

    score = cross_val_score(model, X_train, y_train, cv=5, scoring=scorer)


    return score.mean()

Hyperparameter dapat disesuaikan dengan algoritma yang digunakan. Kali ini kita menggunakan Random Forest sehingga yang dapat kita select adalah *n_estimators, max_depth, min_samples_split,* dan *min_samples_leaf*

In [59]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=92))

[I 2025-10-04 07:37:06,478] A new study created in memory with name: no-name-ea208717-d059-4c94-9cd0-ea9409469d04


In [60]:
study.optimize(objective, n_trials=100)

[I 2025-10-04 07:37:15,206] Trial 0 finished with value: 0.9498015873015874 and parameters: {'n_estimators': 898, 'max_depth': 42, 'min_samples_split': 15, 'min_samples_leaf': 16}. Best is trial 0 with value: 0.9498015873015874.
[I 2025-10-04 07:37:16,051] Trial 1 finished with value: 0.9595238095238094 and parameters: {'n_estimators': 100, 'max_depth': 45, 'min_samples_split': 12, 'min_samples_leaf': 16}. Best is trial 1 with value: 0.9595238095238094.
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.12/dist-packages/sklearn/me

it may take a while... so just wait n see ^^
<br>
they recommend to set n_trials at 100 cz it seems there's no significant score increase after 100 trials (also inefficient too, you'll have to wait in a quite long time)

In [61]:
study.best_params

{'n_estimators': 584,
 'max_depth': 22,
 'min_samples_split': 9,
 'min_samples_leaf': 19}

In [62]:
best_params = study.best_params

Berikut hasil hyperparameter tuning dari Optuna

In [63]:
# cek hasil hyperparameter tuning dari Optuna
import matplotlib.pyplot as plt
optuna.visualization.plot_optimization_history(study)

In [64]:
optuna.visualization.plot_parallel_coordinate(study)

* Garis yang memiliki warna yang lebih tua bekerja dengan lebih baik dibandingkan dengan yang berwarna muda

In [65]:
optuna.visualization.plot_slice(study, params=['n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf'])

* Makin atas makin bagus dan makin tua warnanya attemptnya makin baru

In [66]:
optuna.visualization.plot_param_importances(study)

# Random Forest Using Optuna

In [68]:
# simpan hasil best hyperparameter tuning ke variabel bari
best_n_estimators= best_params['n_estimators']
best_max_depth= best_params['max_depth']
best_min_samples_split= best_params['min_samples_split']
best_min_samples_leaf= best_params['min_samples_leaf']

In [70]:
best_model = RandomForestClassifier(n_estimators=best_n_estimators, max_depth=best_max_depth,
                                    min_samples_split=best_min_samples_split,
                                    min_samples_leaf=best_min_samples_leaf)

best_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=22, min_samples_leaf=19, min_samples_split=9,
                       n_estimators=584)

In [71]:
y_pred = best_model.predict(X_test)

In [73]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.3f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.3f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted'):.3f}")
print(classification_report(y_test, y_pred))

Accuracy: 1.000
Precision: 1.000
Recall: 1.000
F1 Score: 1.000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        11
           2       1.00      1.00      1.00         9

   micro avg       1.00      1.00      1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30
 samples avg       1.00      1.00      1.00        30



Tidak terdapat kenaikan skor dengan sebelum menggunakan Optuna sebab skor yang dihasilkan melalui base model saja sudah bagus